In [1]:
import pandas as pd
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ROBOTA
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_16200\584243297.py:13: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\relia\AppData\Local\Temp\ipykernel_16200\584243297.py:14: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [2]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2024"] != 0]

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [3]:
df_bridges_all["Trieda_PK"].value_counts()

Trieda_PK
cesta III. triedy                      3967
cesta I. triedy                        1913
cesta II. triedy                       1472
diaľnica                                771
miestna neurčená                         56
účelová cesta (UC)                       13
II. trieda - miestna zberná (MZ)          2
III. trieda - miestna obslužná (MO)       1
Name: count, dtype: int64

In [4]:
filtered_bridges_I = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta I. triedy')
]

filtered_bridges_II = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta II. triedy')
]

filtered_bridges_III = df_bridges_all[
    (df_bridges_all["Trieda_PK"] == 'cesta III. triedy')
]

filtered_bridges_I = filtered_bridges_I[filtered_bridges_I["n_2024"] != 0]
filtered_bridges_II = filtered_bridges_II[filtered_bridges_II["n_2024"] != 0]
filtered_bridges_III = filtered_bridges_III[filtered_bridges_III["n_2024"] != 0]

filtered_bridges_I.shape, filtered_bridges_II.shape, filtered_bridges_III.shape

((1913, 44), (1472, 44), (3967, 44))

In [5]:
import plotly.graph_objs as go
def calculate_relative_condition_counts(data, start_year=2012, end_year=2024):
    relative_counts = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        column_name = f"n_{year}"
        year_data = data[data[column_name] > 0]  # Exclude bridges not assessed
        counts = year_data[column_name].value_counts().rename_axis(
            'Condition').reset_index(name=column_name)
        total = counts[column_name].sum()
        counts[column_name] = (counts[column_name] / total) * 100

        if relative_counts.empty:
            relative_counts = counts
        else:
            relative_counts = pd.merge(
                relative_counts, counts, on='Condition', how='outer')

    relative_counts.fillna(0, inplace=True)
    relative_counts.sort_values('Condition', inplace=True)
    return relative_counts


def calculate_absolute_condition_counts(data, start_year=2012, end_year=2024):
    absolute_counts = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        column_name = f"n_{year}"
        year_data = data[data[column_name] > 0]  # Exclude bridges not assessed
        counts = year_data[column_name].value_counts().rename_axis(
            'Condition').reset_index(name=column_name)

        if absolute_counts.empty:
            absolute_counts = counts
        else:
            absolute_counts = pd.merge(
                absolute_counts, counts, on='Condition', how='outer')

    absolute_counts.fillna(0, inplace=True)
    absolute_counts.sort_values('Condition', inplace=True)
    return absolute_counts


def create_plot(relative_counts, title, filename, start_year=2012, end_year=2024, b=500, h=500):
    color_palette = ['#46a7a2', '#e7a564', '#8bae7c',
                     '#353a29', '#eae1a9', '#c25469', '#2d505d']

    # Set up the years range based on start and end year
    years = [f"n_{year}" for year in range(start_year, end_year + 1)]
    display_years = [str(year) for year in range(start_year, end_year + 1)]

    # Adjust data for plotting
    data_for_plotting = relative_counts.set_index(
        'Condition')[years].T.reset_index()
    data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)
    data_for_plotting['Year'] = display_years

    fig = go.Figure()

    for i, condition in reversed(list(enumerate(relative_counts['Condition'].unique()))):
        fig.add_trace(go.Scatter(
            x=data_for_plotting['Year'],
            y=data_for_plotting[condition],
            mode='lines',
            stackgroup='one',
            name=str(condition),
            line=dict(color=color_palette[i % len(color_palette)])
        ))

    fig.update_layout(
        title=title,
        yaxis_title='percentuálne zastúpenie',
        yaxis=dict(ticksuffix="%", range=[0, 100]),
        font=dict(family="Calibri, sans-serif", size=14),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.080,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            tickvals=[str(year) for year in range(start_year, end_year + 1)],
            ticktext=[
                f"'{str(year)[-2:]}" for year in range(start_year, end_year + 1)]
        ),
        plot_bgcolor='#EEEEEE',
        margin=dict(l=0, r=20, t=75, b=25),
        width=b,
        height=h
    )

    fig.show()

In [6]:
relative_condition_counts_II = calculate_relative_condition_counts(
    filtered_bridges_II)

## Relatívne

In [7]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Adjust years to match the column names in the format 'n_2012', 'n_2013', etc.
years = [f"n_{year}" for year in range(2012, 2024)]
# Years as integers for model input
X_train = np.array(range(2012, 2024)).reshape(-1, 1)

predictions = {}

# Use PolynomialFeatures to generate polynomial terms
degree = 3  # You can adjust this to 3 or higher if needed
poly = PolynomialFeatures(degree=degree)

# Transform X_train to include polynomial terms
X_train_poly = poly.fit_transform(X_train)

for condition in relative_condition_counts_II['Condition'].unique():
    # Select condition data from 2012 to 2023 for the current condition
    y_train = relative_condition_counts_II.loc[
        relative_condition_counts_II['Condition'] == condition, years
    ].values.flatten()

    # Train a Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predict for 2024 with polynomial features
    X_test_poly = poly.transform(np.array([[2024]]))
    prediction_2024 = model.predict(X_test_poly)
    predictions[condition] = prediction_2024[0]

# Combine predictions with actual values for comparison
actual_2024 = relative_condition_counts_II.set_index('Condition')['n_2024']
predictions_df = pd.DataFrame({
    'Condition': predictions.keys(),
    'Predicted_2024': predictions.values(),
    'Actual_2024': actual_2024
}).reset_index(drop=True)

print(predictions_df)

   Condition  Predicted_2024  Actual_2024
0          1        2.223245     3.192935
1          2        4.378631     3.328804
2          3       11.194765    16.372283
3          4       41.819102    40.013587
4          5       25.734843    25.067935
5          6       13.485909    11.141304
6          7        1.157138     0.883152


In [11]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Create a copy of the original DataFrame to avoid modifying it
df_predictions = relative_condition_counts_II.copy()

# Extend years to 2035 in the format 'n_2012', 'n_2013', ..., 'n_2035'
years = [f"n_{year}" for year in range(2012, 2027)]
# Prepare X values for model training and predictions (from 2012 to 2035)
X = np.array(range(2012, 2027 + 1)).reshape(-1, 1)

# Initialize PolynomialFeatures with desired degree (adjust as necessary)
degree = 3
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)

# Initialize dictionary to store predictions for each condition
future_predictions = {f"n_{year}": [] for year in range(2025, 2027)}

for condition in df_predictions['Condition'].unique():
    # Train model on the entire data from 2012 to 2024 for the current condition
    y = df_predictions.loc[
        df_predictions['Condition'] == condition,
        [f"n_{year}" for year in range(2012, 2025)]
    ].values.flatten()

    # Train a Linear Regression model with polynomial features
    model = LinearRegression()
    model.fit(X_poly[:len(y)], y)

    # Predict values from 2025 to 2035
    for year in range(2025, 2027):
        X_test_poly = poly.transform(np.array([[year]]))
        future_predictions[f"n_{year}"].append(model.predict(X_test_poly)[0])

# Append predictions to the DataFrame
for year in range(2025, 2027):
    df_predictions[f"n_{year}"] = future_predictions[f"n_{year}"]

# Display the updated DataFrame with predictions for 2025-2035
df_predictions

,Condition,n_2012,n_2013,n_2014,n_2015,n_2016,n_2017,n_2018,n_2019,n_2020,n_2021,n_2022,n_2023,n_2024,n_2025,n_2026
0,1,5.605536,5.605536,5.190311,5.106970,5.175983,4.419890,4.416839,4.315068,3.274216,2.385821,2.453988,2.653061,3.192935,3.411515,4.207007
1,2,9.688581,9.757785,8.996540,8.419600,6.763285,6.146409,5.590062,5.958904,4.229195,5.044308,4.567144,3.877551,3.328804,3.452665,3.367253
2,3,47.474048,47.128028,46.435986,44.858523,42.305038,40.193370,37.681159,35.205479,27.899045,22.835719,19.086571,16.666667,16.372283,13.472029,13.123351
3,4,29.134948,29.134948,30.449827,31.815045,34.851622,37.085635,37.819186,38.287671,42.974079,44.512611,44.239945,42.789116,40.013587,36.749189,31.236781
4,5,6.159170,6.435986,6.505190,7.246377,8.419600,8.632597,10.559006,11.438356,15.347885,17.518746,19.768234,22.244898,25.067935,28.307797,31.485755
5,6,1.799308,1.799308,2.352941,2.484472,2.415459,3.383978,3.864734,4.726027,6.207367,7.634628,9.338787,11.020408,11.141304,13.254310,14.732566
6,7,0.138408,0.138408,0.069204,0.069013,0.069013,0.138122,0.069013,0.068493,0.068213,0.068166,0.545331,0.748299,0.883152,1.356861,1.852280


In [12]:
# Calculate the total sum of the values
total_sum = df_predictions["n_2025"].sum()

# Normalize the values to make the sum 100
df_predictions["n_2025"] = (
    df_predictions["n_2025"] / total_sum) * 100

# Calculate the total sum of the values
total_sum = df_predictions["n_2026"].sum()

# Normalize the values to make the sum 100
df_predictions["n_2026"] = (
    df_predictions["n_2026"] / total_sum) * 100

In [13]:
create_plot(df_predictions, title='Vývoj STS mostov na cestách II. triedy',
            filename='Vyvoj_I', start_year=2012, end_year=2024, b=500, h=500)
create_plot(df_predictions, title='Vývoj STS mostov na cestách II. triedy',
            filename='Vyvoj_I', start_year=2012, end_year=2026, b=575, h=500)